In [1]:
#| default_exp database

# Database Layer
> Local job queue database for chasqui workflow automation

This module manages the local SQLite database that tracks all jobs through their lifecycle.

## Design Principles

- Single source of truth: All job state lives here
- Immutable history: State transitions are logged, never overwritten
- Simple schema: Lightweight, just what we need
- Transaction safety: All writes are atomic

## Job State Machine
```
PREPARED → QUEUED_LOCAL → UPLOADED → SUBMITTED → RUNNING → COMPLETED
                                                         ↘ FAILED
```

## Schema

**jobs table:**
- Tracks individual jobs through their lifecycle
- Primary key: `job_id` (UUID)
- State field tracks current status
- Timestamps for all transitions

**sync_log table:**
- Audit trail of all sync operations
- Helps with debugging and recovery

In [2]:
#| export
import sqlite3
from pathlib import Path
from datetime import datetime
from typing import Optional, List, Dict, Any, Tuple
from contextlib import contextmanager
import json
import uuid
from fastcore.basics import patch

## Database Schema

We keep it minimal but extensible.

In [3]:
#| export

SCHEMA_SQL = """
-- Jobs table: tracks all jobs through their lifecycle
CREATE TABLE IF NOT EXISTS jobs (
    job_id TEXT PRIMARY KEY,
    state TEXT NOT NULL CHECK(state IN (
        'PREPARED', 'QUEUED_LOCAL', 'UPLOADED', 
        'SUBMITTED', 'RUNNING', 'COMPLETED', 'FAILED'
    )),
    
    -- Timestamps
    created_at TEXT NOT NULL,
    queued_at TEXT,
    uploaded_at TEXT,
    submitted_at TEXT,
    completed_at TEXT,
    downloaded_at TEXT,  
    
    -- Local information
    local_path TEXT NOT NULL,
    vasp_config TEXT,  -- JSON blob with VASP parameters
    
    -- Remote information
    remote_path TEXT,
    pbs_id TEXT,  -- PBS job ID (null until submitted)
    
    -- Sync tracking
    last_synced TEXT,
    downloaded INTEGER DEFAULT 0  -- Boolean: results downloaded?
);

-- Index for common queries
CREATE INDEX IF NOT EXISTS idx_jobs_state ON jobs(state);
CREATE INDEX IF NOT EXISTS idx_jobs_pbs_id ON jobs(pbs_id);

-- Sync log: audit trail of all sync operations
CREATE TABLE IF NOT EXISTS sync_log (
    sync_id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT NOT NULL,
    uploaded INTEGER DEFAULT 0,
    submitted INTEGER DEFAULT 0,
    completed INTEGER DEFAULT 0,
    failed INTEGER DEFAULT 0,
    downloaded INTEGER DEFAULT 0,
    details TEXT  -- JSON blob with additional info
);

-- Schema version tracking
CREATE TABLE IF NOT EXISTS schema_version (
    version INTEGER PRIMARY KEY,
    applied_at TEXT NOT NULL
);

INSERT OR IGNORE INTO schema_version (version, applied_at)
VALUES (1, datetime('now'));
"""

## Base class definition

In [4]:
#| export

class ChasquiDB:
    """
    SQLite database manager for chasqui job tracking.
    
    Example:
        >>> db = ChasquiDB("~/.chasqui/jobs.db")
        >>> db.init_db()
        >>> job_id = db.create_job(local_path="/path/to/job")
    """
    
    def __init__(self, db_path: str = "~/.chasqui/jobs.db"):
        """
        Initialize database connection.
        
        Args:
            db_path: Path to SQLite database file
        """
        self.db_path = Path(db_path).expanduser()
        self.db_path.parent.mkdir(parents=True, exist_ok=True)

## Connection Management

We use a context manager to ensure database connections are properly handled.
This pattern ensures commits happen on success and rollbacks on errors.

In [5]:
#| export

@patch
@contextmanager
def _connect(self: ChasquiDB):
    """Context manager for database connections."""
    conn = sqlite3.connect(self.db_path)
    conn.row_factory = sqlite3.Row  # Access columns by name
    try:
        yield conn
        conn.commit()
    except Exception as e:
        conn.rollback()
        raise e
    finally:
        conn.close()

@patch
def init_db(self: ChasquiDB):
    """Initialize database schema."""
    with self._connect() as conn:
        conn.executescript(SCHEMA_SQL)

## Creating Jobs

New jobs start in the `PREPARED` state. They contain:
- A unique UUID identifier
- Path to local VASP inputs
- Optional configuration parameters (stored as JSON)

In [6]:
#| export

@patch
def create_job(
    self: ChasquiDB,
    local_path: str,
    vasp_config: Optional[Dict[str, Any]] = None
) -> str:
    """
    Create a new job in PREPARED state.
    
    Args:
        local_path: Path to local directory with VASP inputs
        vasp_config: Dictionary with VASP parameters (optional)
        
    Returns:
        job_id: UUID for the created job
        
    Example:
        >>> db = ChasquiDB()
        >>> job_id = db.create_job(
        ...     local_path="/scratch/vasp_job_001",
        ...     vasp_config={"encut": 500, "kpoints": [4,4,4]}
        ... )
    """
    job_id = str(uuid.uuid4())
    now = datetime.now().isoformat()
    
    with self._connect() as conn:
        conn.execute("""
            INSERT INTO jobs (job_id, state, created_at, local_path, vasp_config)
            VALUES (?, ?, ?, ?, ?)
        """, (
            job_id,
            'PREPARED',
            now,
            local_path,
            json.dumps(vasp_config) if vasp_config else None
        ))
    
    return job_id

## State Transitions

Jobs move through states as they progress. Each transition:
- Updates the state field
- Records a timestamp
- Can update additional fields (like PBS ID)

The state machine enforces valid transitions through the CHECK constraint.

In [7]:
#| export

@patch
def update_state(
    self: ChasquiDB,
    job_id: str,
    new_state: str,
    **kwargs
):
    """
    Update job state and related fields.
    
    Args:
        job_id: Job UUID
        new_state: New state value
        **kwargs: Additional fields to update (e.g., pbs_id, remote_path)
        
    Example:
        >>> db.update_state("abc-123", "SUBMITTED", pbs_id="12345678")
    """
    # Build UPDATE statement dynamically
    timestamp_field = {
        'QUEUED_LOCAL': 'queued_at',
        'UPLOADED': 'uploaded_at',
        'SUBMITTED': 'submitted_at',
        'COMPLETED': 'completed_at',
        'FAILED': 'completed_at'
    }.get(new_state)
    
    updates = {'state': new_state}
    if timestamp_field:
        updates[timestamp_field] = datetime.now().isoformat()
    updates.update(kwargs)
    
    # Build SQL
    set_clause = ', '.join(f"{k} = ?" for k in updates.keys())
    values = list(updates.values()) + [job_id]
    
    with self._connect() as conn:
        conn.execute(
            f"UPDATE jobs SET {set_clause} WHERE job_id = ?",
            values
        )

## Querying Jobs

Common query patterns for retrieving job information.

In [8]:
#| export

@patch
def get_job(self: ChasquiDB, job_id: str) -> Optional[Dict[str, Any]]:
    """Get job by ID."""
    with self._connect() as conn:
        row = conn.execute(
            "SELECT * FROM jobs WHERE job_id = ?",
            (job_id,)
        ).fetchone()
        return dict(row) if row else None

@patch
def get_jobs_by_state(self: ChasquiDB, state: str) -> List[Dict[str, Any]]:
    """Get all jobs in a given state."""
    with self._connect() as conn:
        rows = conn.execute(
            "SELECT * FROM jobs WHERE state = ?",
            (state,)
        ).fetchall()
        return [dict(row) for row in rows]

@patch
def get_all_jobs(self: ChasquiDB) -> List[Dict[str, Any]]:
    """Get all jobs."""
    with self._connect() as conn:
        rows = conn.execute("SELECT * FROM jobs ORDER BY created_at").fetchall()
        return [dict(row) for row in rows]

## Sync Audit Log

Track all synchronization operations for debugging and monitoring.

In [9]:
#| export

@patch
def log_sync(
    self: ChasquiDB,
    uploaded: int = 0,
    submitted: int = 0,
    completed: int = 0,
    failed: int = 0,
    downloaded: int = 0,  # ADD THIS PARAMETER
    details: Optional[Dict[str, Any]] = None
):
    """
    Record a sync operation.
    
    Args:
        uploaded: Number of jobs uploaded
        submitted: Number of jobs submitted to PBS
        completed: Number of jobs completed
        failed: Number of jobs failed
        downloaded: Number of files downloaded  # ADD THIS LINE
        details: Additional information (optional)
    """
    now = datetime.now().isoformat()
    
    with self._connect() as conn:
        conn.execute("""
            INSERT INTO sync_log 
            (timestamp, uploaded, submitted, completed, failed, downloaded, details)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (
            now,
            uploaded,
            submitted,
            completed,
            failed,
            downloaded,  # ADD THIS VALUE
            json.dumps(details) if details else None
        ))

@patch
def get_last_sync(self: ChasquiDB) -> Optional[Dict[str, Any]]:
    """Get the most recent sync operation."""
    with self._connect() as conn:
        row = conn.execute(
            "SELECT * FROM sync_log ORDER BY sync_id DESC LIMIT 1"
        ).fetchone()
        return dict(row) if row else None

## Tests

Let's verify the database works correctly.

In [10]:
#| hide
import tempfile
import os

# Create temporary database for testing
with tempfile.NamedTemporaryFile(delete=False, suffix='.db') as tmp:
    test_db_path = tmp.name

try:
    # Initialize
    db = ChasquiDB(test_db_path)
    db.init_db()
    print("✓ Database initialized")
    
    # Create job
    job_id = db.create_job(
        local_path="/test/job",
        vasp_config={"encut": 500}
    )
    assert job_id is not None
    print(f"✓ Created job: {job_id}")
    
    # Verify initial state
    job = db.get_job(job_id)
    assert job['state'] == 'PREPARED'
    assert job['local_path'] == '/test/job'
    print("✓ Job created in PREPARED state")
    
    # Update state
    db.update_state(job_id, 'QUEUED_LOCAL')
    job = db.get_job(job_id)
    assert job['state'] == 'QUEUED_LOCAL'
    assert job['queued_at'] is not None
    print("✓ State transition works")
    
    # Query by state
    queued = db.get_jobs_by_state('QUEUED_LOCAL')
    assert len(queued) == 1
    assert queued[0]['job_id'] == job_id
    print("✓ Query by state works")
    
    # Log sync
    db.log_sync(uploaded=1, submitted=0)
    last_sync = db.get_last_sync()
    assert last_sync['uploaded'] == 1
    print("✓ Sync logging works")
    
    print("\n✅ All tests passed!")
    
finally:
    # Cleanup
    os.unlink(test_db_path)

✓ Database initialized
✓ Created job: bb8dd229-bf22-459e-8945-bbe08ea0137b
✓ Job created in PREPARED state
✓ State transition works
✓ Query by state works
✓ Sync logging works

✅ All tests passed!
